In [19]:
import os
import torch
from torch import nn
from torchvision import transforms, datasets
from torch.utils.data import DataLoader,random_split
import datetime

In [20]:
# generate labels set
data_path = os.path.join(os.getcwd(), 'dataset_final/')
label_list = [l.lower().strip() for l in os.listdir(data_path) if not l.startswith('.')]
labels = {ele:i for i, ele in enumerate(label_list)}
print('------ Labels ------', labels, sep='\n')

------ Labels ------
{'dep_curr_src_r0': 0, 'resistor_r1': 1, 'battery_r3': 2, 'gnd_1': 3, 'dep_curr_src_r1': 4, 'battery_r2': 5, 'resistor_r0': 6, 'dc_volt_src_1_r1': 7, 'dep_volt_r3': 8, 'curr_src_r0': 9, 'dc_volt_src_1_r0': 10, 'curr_src_r1': 11, 'dep_volt_r2': 12, 'inductor_r1': 13, 'dc_volt_src_2_r2': 14, 'diode_r0': 15, 'inductor_r0': 16, 'diode_r1': 17, 'dc_volt_src_2_r3': 18, 'dep_curr_src_r3': 19, 'cap_r1': 20, 'battery_r0': 21, 'cap_r0': 22, 'dep_curr_src_r2': 23, 'battery_r1': 24, 'dep_volt_r0': 25, 'curr_src_r3': 26, 'ac_src_r0': 27, 'dc_volt_src_1_r2': 28, 'curr_src_r2': 29, 'dep_volt_r1': 30, 'dc_volt_src_1_r3': 31, 'ac_src_r1': 32, 'dc_volt_src_2_r1': 33, 'diode_r3': 34, 'diode_r2': 35, 'dc_volt_src_2_r0': 36}


In [21]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.00683), std=(0.2440)),
    transforms.Resize((64,64)),
    transforms.Lambda(lambda x: x[0,:,:].T),
])

In [22]:
dataset = datasets.ImageFolder(data_path, transform=transform)
train_data, val_data = random_split(dataset, [int(len(dataset)*0.8), len(dataset)-int(len(dataset)*0.8)])

In [23]:
dataset.classes

['ac_src_r0',
 'ac_src_r1',
 'battery_r0',
 'battery_r1',
 'battery_r2',
 'battery_r3',
 'cap_r0',
 'cap_r1',
 'curr_src_r0',
 'curr_src_r1',
 'curr_src_r2',
 'curr_src_r3',
 'dc_volt_src_1_r0',
 'dc_volt_src_1_r1',
 'dc_volt_src_1_r2',
 'dc_volt_src_1_r3',
 'dc_volt_src_2_r0',
 'dc_volt_src_2_r1',
 'dc_volt_src_2_r2',
 'dc_volt_src_2_r3',
 'dep_curr_src_r0',
 'dep_curr_src_r1',
 'dep_curr_src_r2',
 'dep_curr_src_r3',
 'dep_volt_r0',
 'dep_volt_r1',
 'dep_volt_r2',
 'dep_volt_r3',
 'diode_r0',
 'diode_r1',
 'diode_r2',
 'diode_r3',
 'gnd_1',
 'inductor_r0',
 'inductor_r1',
 'resistor_r0',
 'resistor_r1']

In [24]:
# # calculate mean and std of dataset

# mean = 0.
# std = 0.

# for images, _ in dataset:
#     mean += images.view(1, -1).mean(dim=1)
#     std += images.view(1, -1).std(dim=1)

# mean /= len(dataset)
# std /= len(dataset)

In [25]:
# model class

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.con1 = nn.Conv2d(1, 32, 3, padding=1)
        self.acv1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2)
        self.con2 = nn.Conv2d(32, 64, 3, padding=1)
        self.acv2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2)
        self.con3 = nn.Conv2d(64, 32, 3, padding=1)
        self.acv3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(2)
        self.con4 = nn.Conv2d(32, 16, 3, padding=1)
        self.acv4 = nn.ReLU()
        self.pool4 = nn.MaxPool2d(2)
        self.lin1 = nn.Linear(16*4*4, 1024)
        self.act5 = nn.ReLU()
        self.lin2 = nn.Linear(1024, 37)

    def forward(self, img):
        out = self.pool1(self.acv1(self.con1(img)))
        out = self.pool2(self.acv2(self.con2(out)))
        out = self.pool3(self.acv3(self.con3(out)))
        out = self.pool4(self.acv4(self.con4(out)))
        out = out.view(-1, 16*4*4)
        out = self.act5(self.lin1(out))
        out = self.lin2(out)

        return out


In [26]:
model = CNN()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=5e-2)
train_loader = DataLoader(dataset=train_data, batch_size=32, shuffle=True)

In [27]:
epochs = 100
for epoch in range(epochs + 1):
    loss_train = 0.0
    for imgs, labels in train_loader:
        outputs = model(imgs.unsqueeze(1))
        loss = loss_fn(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_train += loss.item()

    if epoch % 10 == 0:
        print('{} Epoch {}, Training loss {}'.format(datetime.datetime.now(), epoch,loss_train / len(train_loader)))

2023-05-07 11:51:51.057389 Epoch 0, Training loss 2.616977506647996
2023-05-07 12:00:30.942040 Epoch 10, Training loss 0.1082963974162926
2023-05-07 12:08:49.579550 Epoch 20, Training loss 0.0035748853739772783
2023-05-07 12:17:45.574801 Epoch 30, Training loss 0.0003017254136852774
2023-05-07 12:26:17.897856 Epoch 40, Training loss 0.00020504278850947856
2023-05-07 12:35:08.104104 Epoch 50, Training loss 0.0001751331840067883
2023-05-07 12:43:59.113871 Epoch 60, Training loss 0.00016614307228631102
2023-05-07 12:52:39.528572 Epoch 70, Training loss 0.00016014142241617955
2023-05-07 13:01:53.815687 Epoch 80, Training loss 0.00015145274121466888
2023-05-07 13:10:33.198696 Epoch 90, Training loss 0.00014847343132482396
2023-05-07 13:57:45.946182 Epoch 100, Training loss 0.0001456584627678079


In [28]:
val_loader = torch.utils.data.DataLoader(val_data, batch_size=32)

In [29]:
y_pred = []
y_true = []

for name, loader in [('Train', train_loader), ('Validation', val_loader)]:
    correct = 0
    total = 0
    
    with torch.no_grad():
        for imgs, labels in loader:
            outputs = model(imgs.unsqueeze(1))
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            y_pred.extend(predicted)
            y_true.extend(labels)

    print('Accuracy of the network on the {} images: {} %'.format(name, 100 * correct / total))

Accuracy of the network on the Train images: 99.9914486061228 %
Accuracy of the network on the Validation images: 90.35567715458276 %


In [31]:
model.eval()
torch.save(model, 'model.pkl')